In [ ]:
!pip install tensorflow

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.3.0'

In [4]:
# Generated Python module for tensorflow/core/framework/tensor.proto:
tf.core.framework.tensor_pb2

<module 'tensorflow.core.framework.tensor_pb2' from '/usr/local/lib/python3.6/dist-packages/tensorflow/core/framework/tensor_pb2.py'>

In [5]:
# Generated Python module for tensorflow/core/protobuf/saved_model.proto:
tf.core.protobuf.saved_model_pb2

<module 'tensorflow.core.protobuf.saved_model_pb2' from '/usr/local/lib/python3.6/dist-packages/tensorflow/core/protobuf/saved_model_pb2.py'>

In [6]:
# Contents of the generated Python module:
dir(tf.core.protobuf.saved_model_pb2)

['DESCRIPTOR',
 'SavedModel',
 '_SAVEDMODEL',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_b',
 '_descriptor',
 '_message',
 '_reflection',
 '_sym_db',
 '_symbol_database',
 'sys',
 'tensorflow_dot_core_dot_protobuf_dot_meta__graph__pb2']

In [7]:
!wget 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz'
!tar -xzf ssd_mobilenet_v1_coco_2018_01_28.tar.gz
%cd ssd_mobilenet_v1_coco_2018_01_28/

pb_file_name = "saved_model/saved_model.pb"
with open(pb_file_name, "rb") as f:
    binary_data = f.read()

len(binary_data)

--2020-09-28 02:11:42--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.195.128, 2607:f8b0:4001:c11::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.195.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76541073 (73M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_coco_2018_01_28.tar.gz’

ssd_mobilenet_v1_co 100%[===================>]  73.00M   242MB/s    in 0.3s    

2020-09-28 02:11:42 (242 MB/s) - ‘ssd_mobilenet_v1_coco_2018_01_28.tar.gz’ saved [76541073/76541073]

/content/ssd_mobilenet_v1_coco_2018_01_28


29700424

In [8]:
saved_model_proto = tf.core.protobuf.saved_model_pb2.SavedModel.FromString(binary_data)
# print(saved_model_proto)

In [9]:
print(f"{len(saved_model_proto.meta_graphs)} MetaGraphDefs in this SavedModel")
meta_graph_proto = saved_model_proto.meta_graphs[0]
graph_def_proto = meta_graph_proto.graph_def

# print(meta_graph_proto)
# print(graph_def_proto)

1 MetaGraphDefs in this SavedModel


In [10]:
import numpy as np

# Byte swap the
def swap16(x):
  return np.frombuffer(x, dtype=np.uint16).byteswap().tobytes()

def swap32(x):
  return np.frombuffer(x, dtype=np.uint32).byteswap().tobytes()

def swap64(x):
  return np.frombuffer(x, dtype=np.uint64).byteswap().tobytes()

def byte_swap_array(content, bytes_per_elem):
    if bytes_per_elem == 1:
      return content

    elif bytes_per_elem == 2:
      return swap16(content)

    elif bytes_per_elem == 4:
      return swap32(content)

    elif bytes_per_elem == 8:
      return swap64(content)

    else:
      return content

In [11]:
set_allowed = set([tf.string, tf.qint8, tf.quint8, tf.bool, tf.int8, tf.uint8])
set_16bits = set([tf.bfloat16, tf.half, tf.qint16, tf.quint16, tf.uint16, tf.int16])
set_32bits = set([tf.float32, tf.int32, tf.qint32, tf.uint32])
set_64bits = set([tf.int64, tf.uint64, tf.double])

# Separate the tensors according to tf.dtype of their tensor_content
def byte_swap_tensor(orig_tensor) -> bytes:

    datatype = orig_tensor.dtype
    content = orig_tensor.tensor_content
    bytes_per_elem = 0

    if datatype in set_allowed:
      bytes_per_elem = 1
    elif datatype in set_16bits:
      bytes_per_elem = 2
    elif datatype in set_32bits:
      bytes_per_elem = 4
    elif datatype in set_64bits:
      bytes_per_elem = 8
    elif datatype in set_complex64:
      bytes_per_elem = 4
    elif datatype in set_complex128:
      bytes_per_elem = 8
    else:
      print("Byteswapping not supported for datatype")
      return content
  
    return byte_swap_array(content, bytes_per_elem)

In [ ]:
# Iterate over the nodes of graphs in the top-level graph
for n in graph_def_proto.node:
    # print(n.name)
    for attr_name in n.attr:
        attr_value = n.attr[attr_name]
        if attr_value.HasField("tensor"):   
            tensor_value = attr_value.tensor
            if len(tensor_value.tensor_content) > 0:
              print(f"       Before: {attr_name} => {tensor_value.tensor_content}")
              swapped_tensor_content = byte_swap_tensor(tensor_value)
              tensor_value.tensor_content = swapped_tensor_content
              print(f"       After: {attr_name} => {tensor_value.tensor_content}")

In [13]:
# Pass 2: Iterate over the graph of each function in the top-level graph's function library
for f in graph_def_proto.library.function:
    # print(f.signature.name)
    for n in f.node_def:
        # print(f"   {n.name}")
        for attr_name in n.attr:
            attr_value = n.attr[attr_name]
            if attr_value.HasField("tensor"):   
                tensor_value = attr_value.tensor
                if len(tensor_value.tensor_content) > 0:
                    print(f"       Before: {attr_name} => {tensor_value.tensor_content}")
                    swapped_tensor_content = byte_swap_tensor(tensor_value)
                    tensor_value.tensor_content = swapped_tensor_content
                    print(f"       After: {attr_name} => {tensor_value.tensor_content}")                

In [14]:
# Our SavedModel message after modifications
saved_model_proto

In [ ]:
binary_saved_model_proto = graph_def_proto.SerializeToString()
len(binary_saved_model_proto)

In [ ]:
with open("swapped_saved_model.pb", "wb") as f:
    f.write(binary_saved_model_proto)